In [18]:
import xarray as xr
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import pyart
import glob
import multiprocessing
import requests
import shutil


import wradlib as wrl
import gzip
import os
from osgeo import osr

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import fiona
import shapely
import pickle

In [19]:
# A def that emails me when the data is ready
import smtplib
from email.message import EmailMessage

def email_alert(subject, body, to):
    
    msg = EmailMessage()
    msg.set_content(body)
    msg['subject'] = subject
    msg['to'] = to
    
    
    user = 'jeremy.corner.alert@gmail.com'
    password = 'jrdakdhxfmvaigsy'
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(user, password)
    server.send_message(msg)
    
    server.quit()

In [3]:
time1 = datetime.now()


In [4]:
day = '26'
month = '06'
month1 = 'June'
year = '2021'


In [5]:
past = datetime.now()

for hour in range(24):
    if hour <= 9:
        for minute in range(59):
            if minute % 2 == 0:
                if minute <= 8:
                    time = f'0{hour}0{minute}00'
                    url = f'https://mtarchive.geol.iastate.edu/{year}/{month}/{day}/mrms/ncep/PrecipRate/PrecipRate_00.00_{year}{month}{day}-{time}.grib2.gz'
                    r = requests.get(url, allow_redirects=True)
                    filename = url[-41:]
                    if int(r.headers.get('content-length', None)) > 10000:
                        open(f'/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/{filename}', 'wb').write(r.content)
                    
                else:
                    time = f'0{hour}{minute}00'
                    url = f'https://mtarchive.geol.iastate.edu/{year}/{month}/{day}/mrms/ncep/PrecipRate/PrecipRate_00.00_{year}{month}{day}-{time}.grib2.gz'
                    r = requests.get(url, allow_redirects=True)
                    filename = url[-41:]
                    if int(r.headers.get('content-length', None)) > 10000:
                        open(f'/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/{filename}', 'wb').write(r.content)
                    
                    
    else:
        for minute in range(59):
            if minute % 2 == 0:
                if minute <= 8:
                    time = f'{hour}0{minute}00'
                    url = f'https://mtarchive.geol.iastate.edu/{year}/{month}/{day}/mrms/ncep/PrecipRate/PrecipRate_00.00_{year}{month}{day}-{time}.grib2.gz'
                    r = requests.get(url, allow_redirects=True)
                    filename = url[-41:]
                    if int(r.headers.get('content-length', None)) > 10000:
                        open(f'/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/{filename}', 'wb').write(r.content)
                    
                else:
                    time = f'{hour}{minute}00'
                    url = f'https://mtarchive.geol.iastate.edu/{year}/{month}/{day}/mrms/ncep/PrecipRate/PrecipRate_00.00_{year}{month}{day}-{time}.grib2.gz'
                    r = requests.get(url, allow_redirects=True)
                    filename = url[-41:]
                    if int(r.headers.get('content-length', None)) > 10000:
                        open(f'/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/{filename}', 'wb').write(r.content)
                    
                
               

In [6]:
for pathname in glob.glob(f'/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/*.gz'):
    with gzip.open(f'{pathname}', 'rb') as f_in:
        with open(f'{pathname[0:-3]}', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
current = datetime.now()
print(current - past) 


0:03:01.120545


In [7]:
data = glob.glob(f'/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/*.grib2')
data.sort()
#print(data[][-12:-10])

In [103]:
ds = xr.open_dataset(data[0])


#ds = xr.open_dataset(os.path.join(pathfiles,datafile), engine='cfgrib')


#Subset the data and create grid
#ds_sub = ds.sel(latitude=slice(43,40),longitude=slice(270,273))
ds_sub = ds.sel(latitude=slice(45,36),longitude=slice(266,275))


y_rad = ds_sub['latitude'].values
x_rad = ds_sub['longitude'].values

mesh_x, mesh_y = np.meshgrid(x_rad, y_rad)

# Put the two coordinate grids in one array.

grid_xy = np.zeros((900,900,2))

grid_xy[:,:,0] = mesh_x
grid_xy[:,:,1] = mesh_y

In [9]:
# Read the watershed shapefile.
# this file is in UTM
fname_shp = '/gpfs/fs1/home/ac.jcorner/Rainfall/Data/city_detailed_utm/city_detailed_utm.shp'

dataset, inLayer = wrl.io.open_vector(fname_shp)
borders, keys = wrl.georef.get_vector_coordinates(inLayer, key='node_id')

# Define different projections that will be used in the processing.

proj_wgs = osr.SpatialReference()
proj_wgs.ImportFromEPSG(4326)

proj_aeqd = osr.SpatialReference()
proj_aeqd.ImportFromEPSG(54032)

proj_IL = osr.SpatialReference()
proj_IL.ImportFromEPSG(26771)

proj_IL_UTM = osr.SpatialReference()
proj_IL_UTM.ImportFromEPSG(26916)

0

In [10]:
# Reproject the radar grid to UTM, to match the shapefile.
grid_xy_utm = wrl.georef.reproject(grid_xy,
                                projection_source=proj_wgs,
                                projection_target=proj_IL_UTM)

x_rad_utm = grid_xy_utm[:,:,0]
y_rad_utm = grid_xy_utm[:,:,1]

In [102]:
# Create a mask to reduce size.
# Reduce grid size using a bounding box (to enhancing performance)
bbox = inLayer.GetExtent()

buffer = 100.
bbox = dict(left=bbox[0] - buffer, right=bbox[1] + buffer,
            bottom=bbox[2] - buffer, top=bbox[3] + buffer)
mask = (((grid_xy_utm[..., 1] > bbox['bottom']) & (grid_xy_utm[..., 1] < bbox['top'])) &
        ((grid_xy_utm[..., 0] > bbox['left']) & (grid_xy_utm[..., 0] < bbox['right'])))

# Create vertices for each grid cell
# (MUST BE DONE IN NATIVE COORDINATES)

grdverts = wrl.zonalstats.grid_centers_to_vertices(x_rad_utm[mask],
                                                   y_rad_utm[mask], 824,
                                                   824)

In [104]:
past = datetime.now()

file_hour = '00'
grid_p = np.zeros((900,900))

for file in data:  
    if file[-12:-10] == file_hour:
        df = xr.open_dataset(file)
        p_amount = df.sel(latitude=slice(45,36),longitude=slice(266,275))['unknown'].values
        grid_p[:,:] = p_amount/30.0 + grid_p[:,:]
    
    else:
        pd.DataFrame(grid_p).to_csv(f"/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/one_hour_{file_hour}_agg.csv")
        file_hour = file[-12:-10]
        grid_p = np.zeros((900,900))
        df = xr.open_dataset(file)
        p_amount = df.sel(latitude=slice(45,36),longitude=slice(266,275))['unknown'].values
        grid_p[:,:] = p_amount/30.0 + grid_p[:,:]
              
pd.DataFrame(grid_p).to_csv(f"/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/one_hour_{file_hour}_agg.csv")        
current = datetime.now()
print(current - past)    

1:58:33.221985


In [105]:
csv = glob.glob(f'/gpfs/fs1/home/ac.jcorner/Rainfall/Data/MRMS/{day}_{month1}_{year}/*agg.csv')
csv.sort()


In [106]:
def data_agg(data_file_path):
    
    """this is a function built so that MRMS data can be ran in parallel.
    
        data_file_path = the path for the data file
        
        returns csv that aggregates the MRMS file into the catchments."""
    
    past = datetime.now()
    df = pd.read_csv(data_file_path)
    df = df.to_numpy()
    df = df[:,1:]
    
    MRMS_data_agg = []
    
    hour = data_file_path[-10:-8]

    with fiona.open("/gpfs/fs1/home/ac.jcorner/Rainfall/Data/city_detailed_utm/city_detailed_utm.shp") as c:
            for record in c:
                amount = []
                shape = shapely.geometry.shape(record['geometry'])



                for count in range(y_rad_utm[1].size):
                    for i in y_rad_utm[count]:
                        if i >= bbox['bottom'] and i <= bbox['top'] :
                            for j in x_rad_utm[count]:
                                if j >= bbox['left'] and j <= bbox['right']:
                                    point = shapely.geometry.Point(j, i)
                                    if shape.contains(point):
                                        x = np.where(x_rad_utm[count] == j)
                                        y = np.where(y_rad_utm[count] == i)
                                        amount.append(df[x[0],y[0]][0])
                                        

                if len(amount) > 0:
                    maxi = np.max(amount)
                    print(f"catchment {record['id']} with the amount of {maxi} at {hour}.")
                    MRMS_data_agg.append(maxi)

                else:
                    print(f"catchment {record['id']} with the amount of nan at {hour}.")
                    f_amount = np.average(amount)
                    MRMS_data_agg.append(f_amount)

    f = np.array(MRMS_data_agg)
    df = pd.DataFrame(f)
    
    df.to_csv(f'/gpfs/fs1/home/ac.jcorner/Rainfall/MRMS_data_agg_{year}{month}{day}_{hour}.csv')
    
    email_alert('Done, Time To Talk To Scott', f'the script should be done now for {hour}', 'jeremy.corner1998@gmail.com')
    
    current = datetime.now()
    print(current - past)
    
    return MRMS_data_agg

In [107]:
for timestamp in range(len(csv)):
    d = multiprocessing.Process(target=data_agg, args=[csv[timestamp]])
    d.start()

catchment 0 with the amount of 5.200000032782555 at 23.
catchment 0 with the amount of 7.113333227112889 at 22.
catchment 0 with the amount of 7.226666357368231 at 17.
catchment 0 with the amount of 3.8266667015850544 at 10.catchment 0 with the amount of 0.9833333678543568 at 16.

catchment 0 with the amount of 7.4599997801706195 at 18.
catchment 0 with the amount of 5.856666540727019 at 21.
catchment 0 with the amount of 0.0400000028312206 at 13.
catchment 0 with the amount of 11.23666686564684 at 08.
catchment 0 with the amount of 9.253333508968351 at 19.
catchment 0 with the amount of 0.4399999985471368 at 14.
catchment 0 with the amount of 0.1566666662693023 at 12.
catchment 0 with the amount of 13.040000326931477 at 06.
catchment 0 with the amount of 1.893333315849304 at 11.
catchment 0 with the amount of 34.906665824353695 at 00.
catchment 0 with the amount of 10.743333153426647 at 05.
catchment 0 with the amount of 31.023332878947254 at 02.
catchment 0 with the amount of 18.1833

/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


catchment 1 with the amount of nan at 22.
catchment 1 with the amount of nan at 17.


/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


catchment 1 with the amount of nan at 16.


...

/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


catchment 2 with the amount of nan at 23.
catchment 2 with the amount of nan at 16.
catchment 2 with the amount of nan at 17.
catchment 2 with the amount of nan at 22.
catchment 2 with the amount of nan at 18.
catchment 2 with the amount of nan at 10.
catchment 2 with the amount of nan at 19.
catchment 2 with the amount of nan at 11.
catchment 2 with the amount of nan at 21.
catchment 2 with the amount of nan at 08.
catchment 2 with the amount of nan at 13.
catchment 2 with the amount of nan at 06.
catchment 2 with the amount of nan at 12.
catchment 2 with the amount of nan at 09.
catchment 2 with the amount of nan at 05.
catchment 2 with the amount of nan at 14.
catchment 2 with the amount of nan at 00.
catchment 2 with the amount of nan at 01.
catchment 2 with the amount of nan at 02.
catchment 2 with the amount of nan at 07.
catchment 2 with the amount of nan at 03.
catchment 2 with the amount of nan at 15.
catchment 2 with the amount of nan at 04.
catchment 2 with the amount of nan

Process Process-46:
Exception ignored in: Process Process-48:
Exception ignored in: Exception ignored in: <function BaseGeometry.__del__ at 0x2aeba329c550>Process Process-43:
Process Process-41:
<function BaseGeometry.__del__ at 0x2aeba329c550>Process Process-45:
Process Process-33:
Process Process-47:
Process Process-32:
Process Process-31:
Process Process-42:
Process Process-37:
Process Process-36:
Process Process-30:
Process Process-35:
<function BaseGeometry.__del__ at 0x2aeba329c550>
Exception ignored in: Process Process-26:

Process Process-27:
Process Process-28:
Process Process-39:

<function BaseGeometry.__del__ at 0x2aeba329c550>Traceback (most recent call last):
Traceback (most recent call last):
Process Process-25:
Traceback (most recent call last):
  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/geometry/base.py", line 209, in __del__
  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/geometry/base.py", line 20

  File "/tmp/ipykernel_22819/2526830807.py", line 30, in data_agg
    point = shapely.geometry.Point(j, i)



  File "/tmp/ipykernel_22819/2526830807.py", line 28, in data_agg
    for j in x_rad_utm[count]:

  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/geometry/point.py", line 58, in __init__
    self._set_geom(geom)

  File "/tmp/ipykernel_22819/2526830807.py", line 30, in data_agg
    point = shapely.geometry.Point(j, i)
  File "/tmp/ipykernel_22819/2526830807.py", line 30, in data_agg
    point = shapely.geometry.Point(j, i)
  File "/tmp/ipykernel_22819/2526830807.py", line 30, in data_agg
    point = shapely.geometry.Point(j, i)
  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/geometry/point.py", line 58, in __init__
    self._set_geom(geom)
  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/geometry/point.py", line 57, in __init__
    geom, n = geos_point_from_py(tuple(args))
  File "


KeyboardInterrupt
    
super().__setattr__(name, value)  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/geometry/base.py", line 764, in contains
    return bool(self.impl['contains'](self, other))
  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/geos.py", line 575, in errcheck_predicate
    if result == 2:
super().__setattr__(name, value)try:

KeyboardInterrupt
  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/impl.py", line 58, in __getitem__
    return self.map[key]

KeyboardInterruptKeyboardInterruptKeyboardInterrupt
: KeyboardInterrupt: 
: 

Exception ignored in: Process Process-38:
Process Process-44:
<function BaseGeometry.__del__ at 0x2aeba329c550>Process Process-34:
Process Process-40:

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ac.jcorner/anaconda3/envs/data/lib/python3.9/site-packages/shapely/geometry/base.py", line 209, in __del__
  File "/

In [15]:
time2 = datetime.now()
print(time2 - time1)

2:04:36.533618
